In [153]:
# This solution is not working properly

In [117]:
import re, copy, numpy as np
from collections import defaultdict
import networkx as nx
from aoclib import *

In [136]:
class Unit(object):
    def __init__(self, x, y, race):
        self.x = x
        self.y = y
        self.race = race
        self.ap = 3
        self.hp = 200

    def attack(self, other_unit):
        other_unit.hp -= self.ap

    def is_dead(self):
        return self.hp < 1

def get_sorted_units(units):
    l_units = list(units)
    l_units.sort(key = lambda unit: (unit.y, unit.x))
    return l_units

def attackable_enemy(units, unit):
    other_race = 'E' if unit.race == 'G' else 'G'
    attackable = []
    for dir in directions:
        y_new, x_new = unit.y + dir[1], unit.x + dir[0]
        #try:
        if grid[y_new][x_new] == other_race:
            attackable.append(find_unit_by_position(units, x_new, y_new))

    if attackable:
        attackable.sort(key = lambda u: (u.hp, u.y, u.x))
        return attackable[0]
        #except:
        #    pass
    return None

def find_unit_by_position(units, x, y):
    for u in units:
        if u.x == x and u.y == y:
            return u
    return None

def check_combat_ended(units):
    race = None
    for u in units:
        if not race:
            race = u.race
            continue
        if u.race != race:
            return False
    return True

# This function is ought to manipulate 'units', 'grid', and 'graph'
def process_unit_death(units, unit, grid, graph):
    units = units - {unit}
    grid[unit.y][unit.x] = '.'
    for dir in directions:
        y_new, x_new = unit.y + dir[1], unit.x + dir[0]
        if grid[y_new][x_new] == '.':
            graph.add_edge((unit.x, unit.y), (x_new, y_new))
    return units, grid, graph

In [149]:
input = """
  #######
  #.G.E.#         
  #E.G.E#           
  #.G.E.#           
  #######
"""

input = """
#########
#G..G..G#
#.......#
#.......#
#G..E..G#
#.......#
#.......#
#G..G..G#
#########
"""

input = """
#######   
#.G...# 
#...EG#   
#.#.#G#  
#..G#E#   
#.....#   
#######
"""

input_ = """
#######       
#G..#E#      
#E#E.E#     
#G.##.# 
#...#E#     
#...E.#       
####### 
"""

input = """
#######        
#E..EG#     
#.#G.E#   
#E.##E# 
#G..#.#    
#..E#.#    
#######
"""

input = """
#######     
#E.G#.#    
#.#G..#    
#G.#.G#   
#G..#.#   
#...E.#    
####### 
"""

input = open("15.txt").read().strip()

input = input.strip().replace(" ", "")

In [151]:
# Init grid
global grid
grid = [list(row) for row in input.splitlines()]

# Allowed directions
directions = ((0, -1), (-1, 0), (1, 0), (0, 1))

# Initialize units and walls
units = set()
walls = set()
global graph
graph = nx.Graph()
for y, row in enumerate(grid):
    for x, char in enumerate(row):
        if char == '.':
            for dir in directions:
                y_new, x_new = y + dir[1], x + dir[0]
                if grid[y_new][x_new] == '.':
                    graph.add_edge((x, y), (x_new, y_new))
        if char in ['E', 'G']:
            units.add(Unit(x, y, char))
        if char == '#':
            walls.add((x, y))

## Simulation part 1
k = 0
stop = False
while not check_combat_ended(units) and not stop:
    # Simulate one round
    units_ordered = get_sorted_units(units)

    while units_ordered:
        current_unit = units_ordered.pop(0)
        if not units_ordered:
            k += 1
        current_race = current_unit.race
    
        # Add graph edges from current position
        for dir in directions:
            y_new, x_new = current_unit.y + dir[1], current_unit.x + dir[0]
            if grid[y_new][x_new] == '.':
                graph.add_edge((current_unit.x, current_unit.y), (x_new, y_new))
    
        # Save other race
        other_race = 'E' if current_unit.race == 'G' else 'G'
    
        # Attack if in range
        if enemy := attackable_enemy(units, current_unit):
            current_unit.attack(enemy)
            if enemy.is_dead():
                units, grid, graph = process_unit_death(units, enemy, grid, graph)
                if check_combat_ended(units):
                    stop = True
                    break
            continue
        
        # Get target positions and calculate distances
        reachable_target_positions = []
        distances = []
        for other_unit in units - {current_unit}:
            if other_unit.race == other_race:
                x = other_unit.x
                y = other_unit.y
                for dir in directions:
                        y_new, x_new = y + dir[1], x + dir[0]
                        if grid[y_new][x_new] == '.':
                            try:
                                path_length = nx.shortest_path_length(graph, (current_unit.x, current_unit.y), (x_new, y_new))
                                reachable_target_positions.append((x_new, y_new, path_length))
                            except:
                                pass
    
        # Get target and move 1 step, if possible
        if reachable_target_positions:
            reachable_target_positions.sort(key = lambda pos: (pos[2], pos[1], pos[0]))
            
            # Get target
            target = reachable_target_positions[0]
            
            # Move 1 step
            for dir in directions:
                y_new, x_new = current_unit.y + dir[1], current_unit.x + dir[0]
                if grid[y_new][x_new] == '.' and nx.shortest_path_length(graph, (x_new, y_new), (target[0], target[1])) < target[2]:
                    # Set grid to '.'
                    grid[current_unit.y][current_unit.x] = '.'
                    # Move unit
                    current_unit.x = x_new
                    current_unit.y = y_new
                    # Set grid to race
                    grid[current_unit.y][current_unit.x] = current_race
                    # Can break loop here, since moving was successful
                    break
                    
        # Remove any edge for the new or old grid position
        for dir_ in directions:
            y_new_, x_new_ = current_unit.y + dir_[1], current_unit.x + dir_[0]
            try:
                graph.remove_edge((current_unit.x, current_unit.y), (x_new_, y_new_))
            except:
                pass
    
        # Attack if in range
        if enemy := attackable_enemy(units, current_unit):
            current_unit.attack(enemy)
            if enemy.is_dead():
                units, grid, graph = process_unit_death(units, enemy, grid, graph)
                if check_combat_ended(units):
                    stop = True
                    break

    if k % 10 != 0:
        continue
    
    print("After round", k)
    for y in range(len(grid)):
        str_ = ''.join(grid[y])
        for x in range(len(grid[y])):
            if grid[y][x] in ['E', 'G']:
                unit = find_unit_by_position(units, x, y)
                str_ += "  (" + str(unit.hp) + ") "
        print(str_)

    if k == 100:
        break
          
    #print(list2string(grid))

print("Combat ended after", k, "rounds")
print("Outcome: ", k * sum([u.hp for u in units]))
print("HP: ", sum([u.hp for u in units]), [u.hp for u in units])
print(list2string(grid))

After round 10
################################
########.#######################
#######..##..###################
#######.....##.#################
#######....##.....##############
######.......G..#.##############  (200) 
######...........GE....#########  (176)   (158) 
#######.........G.GE.#.#########  (200)   (188)   (194) 
#####.#....G..G..G.G.#..########  (200)   (200)   (200)   (200) 
####.........G...#.###.#########  (200) 
####...............####.########
#####...................####.###
##.#......G...#####......##..###  (200) 
#............#######......#.####
##..........#########....##..###
#........G..#########..G.....###  (200)   (200) 
#........G#.#########..EG##...##  (200)   (137)   (200) 
#...##.G..#.#########..GE.....##  (200)   (122)   (200) 
########...G#########..E......##  (200)   (200) 
#########....#######........E..#  (200) 
#######.......#####........E..##  (200) 
##########....................##
##########..G.....##....#.....##  (200) 
#########..GEG........####

In [125]:
def test_attack_enemy():
    unita = Unit(0, 0, 'E')
    unitb = Unit(0, 1, 'G')
    hp_before = unitb.hp
    unita.attack(unitb)
    assert unitb.hp == hp_before - 3

# Execute tests
test_attack_enemy()

In [152]:
222025 - 220480

1545